<a href="https://colab.research.google.com/github/2kdatawizard/data_engineering_interviews/blob/main/de_interview_kit_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1

In [ ]:
# Exercise 1
def create_functions():
    function_list = []
    for i in range(5):
        function_list.append(lambda: i)
    return function_list

functions = create_functions()
results = [f() for f in functions]
print(results)

[4, 4, 4, 4, 4]


# Questions
1. What will be the output of this code?
2. Why does it behave this way?
3. How would you modify the code to make it
4. produce the expected result of [0, 1, 2, 3, 4]?
"""

# Problem 2

In [ ]:
# Exercise 2
def process_large_dataset(data_source, batch_size=1000):
    results = []

    for i in range(0, len(data_source), batch_size):
        batch = data_source[i:i+batch_size]
        processed = [item * 2 for item in batch]
        results.extend(processed)

    return results

# Usage
large_data = list(range(10000000))  # 10 million items
processed_data = process_large_dataset(large_data)

# Questions

1. What potential issue might occur when running this code with very large datasets?
2. Rewrite the function to be memory-efficient using generators and the yield keyword.
3. If the data_source is actually a generator itself (not a list), would your solution
still work? Why or why not?
4. Explain how your solution improves memory usage compared to the original code.

# Problem 3
You are working with a large dataset containing user activity logs for an e-commerce website. The data is available as a PySpark DataFrame with the following schema:

```
root
 |-- user_id: string
 |-- session_id: string
 |-- timestamp: timestamp
 |-- page_id: string
 |-- action: string
 |-- product_id: string
 |-- category_id: string
 |-- price: double
 |-- quantity: integer

```
Each row represents a user action (view, add_to_cart, purchase, remove_from_cart) on a product page.

## Write PySpark code to:

1. Calculate the conversion rate (percentage of product views that resulted in purchases) for each product category, but only include users who had at least 3 sessions
2. For each user, find the average time between viewing a product and adding it to cart
3. Identify the top 5 products that are most frequently abandoned (added to cart but never purchased in the same session)
4. Create a user engagement metric that combines total time spent, number of actions, and purchase value, then segment users into 'High', 'Medium', and 'Low' engagement groups
5. Optimize your solution for performance, considering data skew, shuffle operations, and caching strategies

## Expected Solution Components
Your solution should include:

1. Import statements and initialization of SparkSession
2. Any necessary data preprocessing steps
3. Implementation of all required analyses using PySpark DataFrame/SQL API
4. Explanation of performance optimization choices
5. Sample code to write results to the appropriate storage format

## Example Data Pattern
```
user_id: "u123", session_id: "s456", timestamp: "2023-09-15 14:32:21",
page_id: "p789", action: "view", product_id: "prod123", category_id: "electronics",
price: 599.99, quantity: null
```

```
user_id: "u123", session_id: "s456", timestamp: "2023-09-15 14:35:42",
page_id: "p789", action: "add_to_cart", product_id: "prod123", category_id: "electronics",
price: 599.99, quantity: 1
```

```
user_id: "u123", session_id: "s456", timestamp: "2023-09-15 14:45:12",
page_id: "checkout", action: "purchase", product_id: "prod123", category_id: "electronics",
price: 599.99, quantity: 1
```

In [25]:
# Step 1: Install PySpark and Java in Colab
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark==3.2.0 findspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 5.6 MB/s e

In [23]:
# Step 2: Configure environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

# Step 3: Import necessary libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, DoubleType, IntegerType
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

AttributeError: 'DataFrameReader' object has no attribute 'cache'